In [2]:
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
%cd ..

FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo
fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
Processing /content/transformers
  Created wheel for transformers: filename=transformers-2.5.1-cp36-none-any.whl size=534520 sha256=bc1d630b10f6ae3621e86f76e634eb58f292cd26003b00633b5ec348b45a26f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-f8wqu3mh/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 2.5.1
    Uninstalling transformers-2.5.1:
      Successfully uninstalled transformers-2.5.1
/content


In [3]:
from bertviz import head_view
from transformers import BartForConditionalGeneration, BartTokenizer

In [0]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [11]:
model = BartForConditionalGeneration.from_pretrained("bart-large-cnn", output_attentions=True, output_past=True)
tokenizer = BartTokenizer.from_pretrained("bart-large")

text = "Seven civilians were killed Monday from two separate roadside mines in Afghanistan's eastern Ghazni province"
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=False)

# token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids)[-1]

# attention = model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()

head_view(attention, tokens)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

На самом деле, в этом случае довольно тяжело интерпретировать активации внимания. Возьмем еще один пример для сравнения. Если смотреть на другие слои, то лучше от этого не становится.


In [12]:
text = "The two New York police officers were shot to death earlier this month in Brooklyn"
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=False)

# token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids)[-1]

# attention = model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()

head_view(attention, tokens)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Здесь уже более явно видны зависимости. На нулевом слое улавливается более широкий контекст. На первом слое уже видны попарные зависимости между токенами. Например, Brookly сильнее зависит от токенов New и York, а police и officers оба реагируют друг на друга.